In [ ]:
!pip install pyarrow==<15
!pip install ibis-framework

In [ ]:
!pip install transformers
!pip install transformers[sentencepiece]
!pip install datasets
!pip install transformers[torch]

In [ ]:
!pip install accelerate -U
!pip install accelerate>=0.20.1

In [ ]:
!wget https://raw.githubusercontent.com/MarCaramani/MK_Thesis/master/THESIS/Dataset.json
!wget https://raw.githubusercontent.com/MarCaramani/MK_Thesis/master/THESIS/Train_Dataset.json

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [ ]:
checkpoint ="facebook/bart-large-cnn"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [ ]:
max_input_length = 1042
max_target_length = 512

def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["TEXT"],
        max_length=max_input_length,
        truncation=True,
    )
    labels = tokenizer(
        examples["SUMMARIZATION"], max_length=max_target_length, truncation=True
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
my_dataset_test = load_dataset("json", data_files="Dataset.json", field="documents")
my_dataset_train = load_dataset("json", data_files="Train_Dataset.json", field="documents")

In [ ]:
my_dataset_test = my_dataset_test.map(preprocess_function, batched=True)
my_dataset_train = my_dataset_train.map(preprocess_function, batched=True)

In [ ]:
from transformers import Trainer

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    logging_dir="./logs",
    logging_steps=10,
    save_steps=10_000,
    save_total_limit=2
)

In [ ]:
train_dataset = my_dataset_train
eval_dataset = my_dataset_test

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": accuracy_score(labels, predictions)}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset= train_dataset,
    eval_dataset= eval_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()